In [46]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

# Udahub Application

## Core Database

**Init DB**

In [47]:
udahub_db = "data/core/udahub.db"

In [48]:
reset_db(udahub_db)

✅ Removed existing data/core/udahub.db
2025-12-03 13:49:14,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-03 13:49:14,470 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/core/udahub.db with fresh schema


In [49]:
engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(bind=engine)

**Account**

In [50]:
account_id = "cultpass"
account_name = "CultPass Card"

In [51]:
with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

## Integrations

**Knowledge Base**

In [52]:
cultpass_articles = []

with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

In [53]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

In [54]:
if len(cultpass_articles) < 14:
    raise AssertionError("You should load the articles with at least 14 records")

In [55]:
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb) 
    

**Ticket**

In [56]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [57]:
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

In [58]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()

    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )

    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=ticket_info["role"],
        content=ticket_info["content"],
    )

    session.add_all([user, ticket, metadata, first_message])


# Tests

In [59]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

<Account(account_id='cultpass', account_name='CultPass Card')>


In [60]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    for article in account.knowledge_articles:
        print(article)

<Knowledge(article_id='8a68f705-8ff5-4f26-88af-cd0539fbd85f', title='How to Reserve a Spot for an Event')>
<Knowledge(article_id='a884508e-a40a-4279-8564-4911e08770c9', title='What's Included in a CultPass Subscription')>
<Knowledge(article_id='c5f010d9-4000-42b4-a137-758a85920227', title='How to Cancel or Pause a Subscription')>
<Knowledge(article_id='8b3d1cd4-6f82-470f-a72e-8f57ab643341', title='How to Handle Login Issues?')>
<Knowledge(article_id='00d48632-d803-4c00-a17f-ba3a50e80ecc', title='Troubleshooting App Crashes or Freezes')>
<Knowledge(article_id='774687aa-4080-405e-b253-d17ad9384ec5', title='Why an Experience Shows 'No Slots Available'')>
<Knowledge(article_id='78267d8e-2908-4f6c-8e80-344b8ddb04fe', title='How to Update Email or Personal Information')>
<Knowledge(article_id='6235772b-4e21-47f9-89b2-33a69cc348ab', title='Payment Failed or Card Declined')>
<Knowledge(article_id='ecb045cf-bb41-4164-84df-839497d3472c', title='Understanding Premium Experiences')>
<Knowledge(art

In [61]:
with get_session(engine) as session:
    core_users = []
    for u in cultpass_users:
        # Check if already exists 
        existing = session.query(udahub.User).filter_by(
            account_id=account_id,
            external_user_id=u["id"]
        ).first()

        if existing:
            continue  # skip if already inserted

        core_users.append(
            udahub.User(
                user_id=str(uuid.uuid4()),
                account_id=account_id,
                external_user_id=u["id"],
                user_name=u["name"]
            )
        )

    session.add_all(core_users)

print(f"Inserted {len(core_users)} new users into UDA-Hub core DB.")

Inserted 5 new users into UDA-Hub core DB.


In [62]:
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

<User(user_id='bbed30b6-4ffe-48de-9c36-309789bacece', user_name='Alice Kingsley', external_user_id='a4ab87')>
<User(user_id='d3ec0586-d56b-42f1-9dcf-065f23f0c4f8', user_name='Bob Stone', external_user_id='f556c0')>
<User(user_id='07b57f04-c18a-4525-9f10-72af57a6f601', user_name='Cathy Bloom', external_user_id='88382b')>
<User(user_id='4a1d9230-cd13-4a20-a1ba-080d11a7d7ae', user_name='David Noir', external_user_id='888fb2')>
<User(user_id='e92c565e-b553-42ea-b422-0093384af5c1', user_name='Eva Green', external_user_id='f1f10d')>
<User(user_id='1fcfe953-4e5c-473b-82f8-1e721b7b5118', user_name='Frank Ocean', external_user_id='e6376d')>


In [63]:
# 10 additional sample tickets with varied issues & owners

additional_tickets_info = [
    {
        "status": "open",
        "content": "I was charged twice for my CultPass subscription this month.",
        "owner_id": cultpass_users[1]["id"],
        "owner_name": cultpass_users[1]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "billing, duplicate charge, payment",
    },
    {
        "status": "open",
        "content": "The app keeps crashing whenever I try to open the experiences tab.",
        "owner_id": cultpass_users[2]["id"],
        "owner_name": cultpass_users[2]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "technical, crash, app",
    },
    {
        "status": "open",
        "content": "Why does the event I want to attend show no slots available?",
        "owner_id": cultpass_users[3]["id"],
        "owner_name": cultpass_users[3]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "reservation, slots, availability",
    },
    {
        "status": "open",
        "content": "My QR code at the venue would not scan and I couldn't get in.",
        "owner_id": cultpass_users[4]["id"],
        "owner_name": cultpass_users[4]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "qr, entry, technical, events",
    },
    {
        "status": "open",
        "content": "I attended an event yesterday but my monthly quota did not update.",
        "owner_id": cultpass_users[5]["id"],
        "owner_name": cultpass_users[5]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "quota, attendance, subscription",
    },
    {
        "status": "open",
        "content": "There are almost no experiences available in my city. Is my region supported?",
        "owner_id": cultpass_users[0]["id"],
        "owner_name": cultpass_users[0]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "region, location, availability",
    },
    {
        "status": "open",
        "content": "I want to upgrade my CultPass plan. How can I change my subscription?",
        "owner_id": cultpass_users[1]["id"],
        "owner_name": cultpass_users[1]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "subscription, upgrade, plan",
    },
    {
        "status": "open",
        "content": "I cancelled my reservation but I am not sure if my quota was restored.",
        "owner_id": cultpass_users[2]["id"],
        "owner_name": cultpass_users[2]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "reservation, cancellation, quota",
    },
    {
        "status": "open",
        "content": "I paid extra for a premium experience. How do I confirm it is still booked?",
        "owner_id": cultpass_users[3]["id"],
        "owner_name": cultpass_users[3]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "premium, payment, reservation",
    },
    {
        "status": "open",
        "content": "I would like a refund because the event was cancelled by the organizer.",
        "owner_id": cultpass_users[4]["id"],
        "owner_name": cultpass_users[4]["name"],
        "role": "user",
        "channel": "chat",
        "tags": "refund, cancellation, billing",
    },
]


In [64]:
with get_session(engine) as session:
    new_tickets = []

    for info in additional_tickets_info:
        # Find or create internal user mapped to external Cultpass user
        user = session.query(udahub.User).filter_by(
            account_id=account_id,
            external_user_id=info["owner_id"],
        ).first()

        if not user:
            user = udahub.User(
                user_id=str(uuid.uuid4()),
                account_id=account_id,
                external_user_id=info["owner_id"],
                user_name=info["owner_name"],
            )
            session.add(user)

        ticket = udahub.Ticket(
            ticket_id=str(uuid.uuid4()),
            account_id=account_id,
            user_id=user.user_id,
            channel=info["channel"],
        )

        metadata = udahub.TicketMetadata(
            ticket_id=ticket.ticket_id,
            status=info["status"],
            main_issue_type=None,       
            tags=info["tags"],
        )

        first_message = udahub.TicketMessage(
            message_id=str(uuid.uuid4()),
            ticket_id=ticket.ticket_id,
            role=info["role"],
            content=info["content"],
        )

        session.add_all([ticket, metadata, first_message])
        new_tickets.append(ticket)

print(f"Inserted {len(additional_tickets_info)} additional tickets.")


Inserted 10 additional tickets.


In [65]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket:udahub.Ticket = user.tickets[0]
    for message in ticket.messages:
        print(message)

<TicketMessage(message_id='bc8c9bbd-08bf-476e-9e5e-2ae2dd827087', role='user', content='I can't log in to my Cultpass ...')>


In [66]:
with get_session(engine) as session:
    print("Total tickets:", session.query(udahub.Ticket).count())
    print("Total ticket messages:", session.query(udahub.TicketMessage).count())
    print("Total ticket metadata rows:", session.query(udahub.TicketMetadata).count())


Total tickets: 11
Total ticket messages: 11
Total ticket metadata rows: 11
